In [3]:
import serial
import datetime
import threading
import time
import queue

In [4]:
frame_w = int(768 / 2)
frame_h = int(592 / 2)
frame = bytearray(frame_w * frame_h)

In [3]:
# Make sure to run this script before connecting the MCU USB (Keep the UART USB connected, so the port doesn't close.)
# Max baudrate = 3000000
serialPort = serial.Serial(port = "COM6", baudrate = 3000000, bytesize = 8, timeout = 1, stopbits = serial.STOPBITS_ONE)
t_before = datetime.datetime.now()

In [4]:
ppf = serialPort.readinto(frame)

t_after = datetime.datetime.now()
t_diff = (t_after - t_before).total_seconds()

In [5]:
print(frame)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [8]:
exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, counter):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter = counter
   def run(self):
      print("Starting " + self.name)
      print_time(self.name, 5, self.counter)
      print("Exiting " + self.name)

def print_time(threadName, counter, delay):
   while counter:
      if exitFlag:
         threadName.exit()
      time.sleep(delay)
      print("%s: %s" % (threadName, time.ctime(time.time())))
      counter -= 1

# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

print("Exiting Main Thread")

Starting Thread-1
Starting Thread-2Exiting Main Thread

Thread-1: Wed Oct  7 19:52:09 2020
Thread-1: Wed Oct  7 19:52:10 2020
Thread-2: Wed Oct  7 19:52:10 2020
Thread-1: Wed Oct  7 19:52:11 2020
Thread-2: Wed Oct  7 19:52:12 2020Thread-1: Wed Oct  7 19:52:12 2020

Thread-1: Wed Oct  7 19:52:13 2020
Exiting Thread-1
Thread-2: Wed Oct  7 19:52:14 2020
Thread-2: Wed Oct  7 19:52:16 2020
Thread-2: Wed Oct  7 19:52:18 2020
Exiting Thread-2


In [ ]:
exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, q):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.q = q
   def run(self):
      print "Starting " + self.name
      process_data(self.name, self.q)
      print "Exiting " + self.name

def process_data(threadName, q):
   while not exitFlag:
      queueLock.acquire()
         if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print "%s processing %s" % (threadName, data)
         else:
            queueLock.release()
         time.sleep(1)

threadList = ["Thread-1", "Thread-2", "Thread-3"]
nameList = ["One", "Two", "Three", "Four", "Five"]
queueLock = threading.Lock()
workQueue = Queue.Queue(10)
threads = []
threadID = 1

# Create new threads
for tName in threadList:
   thread = myThread(threadID, tName, workQueue)
   thread.start()
   threads.append(thread)
   threadID += 1

# Fill the queue
queueLock.acquire()
for word in nameList:
   workQueue.put(word)
queueLock.release()

# Wait for queue to empty
while not workQueue.empty():
   pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
   t.join()
print "Exiting Main Thread"